In [141]:
import os
import pandas as pd

In [162]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"

In [159]:
import requests
import json

In [167]:
def auth():
    return os.getenv('TWITTER_TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):
    
    ttid = 1184334528837574656
    search_url = f"https://api.twitter.com/2/users/{ttid}/tweets" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'public_metrics,created_at',
                    'next_token': {}}
    
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [170]:
start_time = "2021-12-01T00:00:00.000Z"
end_time = "2021-12-22T00:00:00.000Z"

max_results = 100
url = create_url(keyword, start_time,end_time, max_results)
url

('https://api.twitter.com/2/users/1184334528837574656/tweets',
 {'start_time': '2021-12-01T00:00:00.000Z',
  'end_time': '2021-12-22T00:00:00.000Z',
  'max_results': 100,
  'tweet.fields': 'public_metrics,created_at',
  'next_token': {}})

In [183]:
import collections
import time

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

data = [flatten(i) for i in data["data"]]

In [190]:
data = pd.DataFrame(data)
data.sort_values("public_metrics_like_count", ascending=False).head()

,created_at,text,id,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
40,2021-12-03T16:06:09.000Z,"🥳🏆 Conquistamos o 1º lugar na categoria ""Soluç...",1466800936643436545,6,8,144,9
20,2021-12-10T15:31:26.000Z,#DataSus fora do ar?😬\n\nPela BD você pode ace...,1469328913239359499,13,2,82,4
6,2021-12-16T15:43:51.000Z,Escolha o lado prático da vida💚\n\nPela BD voc...,1471506367592628234,5,1,32,0
47,2021-12-01T15:56:11.000Z,📰💡#FicaADica para jornalistas de dados! \n\nA ...,1466073652642947086,5,1,30,0
26,2021-12-09T15:38:08.000Z,💉📊Como está o cenário da vacinação contra COVI...,1468968212784054276,6,1,24,1


In [216]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"

start_list = [f'2021-{i if i>9 else "0"+str(i)}-01T00:00:00.000Z' for i in range(1,13)]
end_list = [f'2021-{i if i>9 else "0"+str(i)}-{31 if i in (1,3,5,7,8,10,12) else 28 if i == 2 else 30}T00:00:00.000Z' for i in range(1,13)]

max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
df = pd.DataFrame()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                df = df.append([flatten(i) for i in json_response["data"]], ignore_index=True)
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                df = df.append([flatten(i) for i in json_response["data"]], ignore_index=True)
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2021-01-01T00:00:00.000Z
Total # of Tweets added:  18
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2021-02-01T00:00:00.000Z
Total # of Tweets added:  87
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  7140dibdnow9c7btw3w3j1tyv2ti3gpcjfgimo245r779
Start Date:  2021-03-01T00:00:00.000Z
Total # of Tweets added:  187
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  7140dibdnow9c7btw3w3ybr5td795ycq1zf4yyh5uvpkg
Start Date:  2021-04-01T00:00:00.000Z
Total # of Tweets added:  287
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  7140dibdnow9c7btw3w4sc9m9alv9pb049n4utwio0gmu
Start Date:  2021-05-01T00:00:00.000Z
Total # of Tweets added:  387
-------------------
-------------------
Token:  None
Endpoin

In [217]:
df["created_at"].max()

'2021-12-22T15:49:40.000Z'

In [218]:
df["created_at"].min()

'2021-01-04T10:47:35.000Z'

In [219]:
df.head()

,id,text,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,created_at
0,1354513520193277953,Participem das nossas comunidades:\n\n- Discor...,0,0,1,0,2021-01-27T19:36:03.000Z
1,1354513519094321152,Batemos R$1000 por mês no financiamento coleti...,3,1,15,1,2021-01-27T19:36:03.000Z
2,1354151419549368321,RT @Mateus_AMaciel: O trabalho que eu levava h...,6,0,0,0,2021-01-26T19:37:11.000Z
3,1353017612607709184,Segundo workshop BD sobre aplicações em R entr...,7,2,49,4,2021-01-23T16:31:51.000Z
4,1352307992293564417,Marquem na agenda: segundo workshop Base dos D...,13,3,81,3,2021-01-21T17:32:04.000Z


In [237]:
pd.set_option('display.max_colwidth', None)
df["created_date"] = pd.to_datetime(df["created_at"]).dt.date
df.sort_values("public_metrics_like_count", ascending=False)[["public_metrics_like_count", "created_date", "text"]].reset_index(drop=True).head(10)

,public_metrics_like_count,created_date,text
0,754,2021-01-04,Estamos mudando a forma de estudar mercado de trabalho no Brasil. 📢\n\nSubimos os microdados completos da RAIS para 1985-2019 na plataforma BD+! São 260 GB prontos para análise. E também agregações por município e UF.\n\nAqui https://t.co/skWWge8Chj\n\n#EconTwitter #research #opendata
1,731,2021-03-26,"Enquanto o o Censo é desestruturado, por aqui vamos na direção oposta: acabamos de liberar os MICRODADOS COMPLETOS do Censo 1970 a 2010.\n\nTrabalhando para universalizar o acesso a dados no Brasil.\n\nBaixe tudo com apenas 3 linhas de código!\n\nAqui: https://t.co/CxCQfX2fYt https://t.co/c5L0FQ2K2f"
2,608,2021-04-26,Prefere fazer suas análises em R? Agora você pode acessar nossos dados usando sua linguagem preferida! 👨‍💻\n \nIsso mesmo! Estamos lançando a versão do pacote da Base dos Dados em R.
3,303,2021-07-19,"Como consultar os 250GB de dados da RAIS, sem baixar nada?🤔\n\nOs microdados da RAIS já estão disponíveis na BD+ para você acessar só com uma consulta de SQL no BigQuery. Preparamos também um tutorial para mostrar como explorar essa base usando nosso datalake público. Veja no 🧶"
4,261,2021-10-11,🎉 We're proud to be named one of the 5 world-wide Google Cloud Customer Award winner for Social Impact! \n\n🇧🇷 Estamos honrados(as) de ser uma das 5 organizações vencedoras do Google Cloud Customer Award na categoria de Impacto Social ao redor do mundo! #bdmais #GoogleCloudNext https://t.co/hIeRyt3mQ4
5,244,2021-06-03,"Os dados da mais importante pesquisa estatística educacional brasileira já estão disponíveis na BD+ para facilitar sua análise! 👨‍🏫\n\nVocê encontra dados do Censo Escolar já integrados, limpos e organizados em nosso data lake público! Veja mais no fio🧶 https://t.co/Dtk4hX6mv9"
6,228,2021-06-25,"Que tal nos ajudar em uma campanha para mapear bases de dados LGBTQIA+? 🏳️‍🌈👩‍💻\n\nIsso mesmo, queremos mapear, tratar e disponibilizar bases de dados que tragam informações sobre a comunidade LGBTQIA+ para facilitar o acesso e análise desses dados. Veja no fio como você pode ajudar. https://t.co/lsYQlEeJfU"
7,184,2021-10-01,NOTA: Estamos encerrando nossa parceria com o DataCamp. \n\nVocê pode conferir os detalhes na nota da imagem e links referentes ao assunto no fio. https://t.co/0JvRxeWUsJ
8,160,2021-04-26,Quer acessar e analisar nossos dados direto pelo ambiente da linguagem? \n\nO pacote já está disponível no CRAN: https://t.co/oFOgwTdHqq
9,154,2021-09-13,"Estava ansioso(a) para o lançamento do nosso novo site, databaser? 😏\n\n📢 Finalmente chegou o GRANDE momento: O NOVO SITE JÁ ESTÁ NO AR! 🎉\n\nCorre lá para explorar as novas funcionalidades, o visual renovado, interface mais intuitiva e MUITO mais! https://t.co/pLFWCheQW9"


In [236]:
df[df["text"].str.lower().str.contains("censo escolar")].sort_values("public_metrics_like_count", ascending=False)[["public_metrics_like_count", "created_date", "text"]].reset_index(drop=True)

,public_metrics_like_count,created_date,text
0,244,2021-06-03,"Os dados da mais importante pesquisa estatística educacional brasileira já estão disponíveis na BD+ para facilitar sua análise! 👨‍🏫\n\nVocê encontra dados do Censo Escolar já integrados, limpos e organizados em nosso data lake público! Veja mais no fio🧶 https://t.co/Dtk4hX6mv9"
1,55,2021-06-04,"Os dados do Censo Escolar já estão disponíveis em nosso datalake público e nós preparamos um tutorial para apresentar as tabelas da base, algumas de suas variáveis e pontos de contato interessantes com outras tabelas disponíveis na BD+ 👩‍🏫\n\nVeja mais no fio🧶 https://t.co/YB5jMglrxC"
2,40,2021-11-15,"👨‍🏫🔍Que tal explorar os dados da mais importante pesquisa estatística educacional do Brasil? \n\nConfira esse artigo que ensina como acessar os microdados do Censo Escolar, com aplicações e pontos de contato com outras tabelas da BD+.\n\n➡️ Acesse por aqui: https://t.co/BMvVwquWJm"
3,23,2021-06-03,"O Censo Escolar, coordenado pelo @inep_oficial, é o principal instrumento de coleta de informações da educação básica no Brasil. Ele abrange as diferentes etapas e modalidades da educação, ou seja, compreende desde a educação infantil até cursos técnicos e profissionalizantes."
4,13,2021-09-24,Os dados do Censo Escolar possibilitaram a elaboração desse mapa que traz um panorama da diferença do perfil racial entre escolas públicas e privadas no Brasil. https://t.co/f5tvO5UjwC
5,9,2021-12-14,"Vale lembrar, pela BD você acessa dados do Censo Escolar já tratados, compatibilizados e prontos para sua análise!😉\n\n➡️Acesse já essa base: https://t.co/NcqYPRuJpA"
6,6,2021-04-15,"➡️ Qual a base de dados que você mais gosta?\n\nOs dados do Censo Escolar tem um lugar especial no meu coração. São dados anuais descritivos de todas as matrículas, escolas e redes do Brasil, que fundamentam o acompanhamento e pesquisa da educação brasileira junto ao SAEB."
7,5,2021-06-04,"Se você já conhece essa base e quer partir para a análise, acesse os dados do Censo Escolar por aqui: https://t.co/yaQPckUG7y"
8,4,2021-04-15,➡️ Conta um pouco sobre um projeto legal que você fez com dados públicos\n\nComo projeto final da graduação fiz uma análise de evasão escolar no município do Rio com os dados do Censo Escolar. Contínua no thread
9,4,2021-06-04,"O Censo Escolar é uma pesquisa anual em que as escolas do país respondem perguntas sobre suas turmas, professores e alunos. Os dados de 2009 até 2020 já estão integrados na BD+. Você já pode abrir as tabelas no R, no Python ou baixá-las via BigQuery."
